# Life expectancy analysis

Below is an analysis predicting the life expectancy of 153 countries.

* Data sources
* Number of countries
* See if there is additional information I should include.

## Importing the required packages

Numpy, pandas and sklear are all needed for this analysis. These are imported below.

In [ ]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from sklearn.linear_model import Lasso

## Extracting the data for this analysis

Sixteen predictors were considered for this analysis, of which 12 were ultimately used. For each of the variables, the latest year with the most complete data that was available were selected for the analysis. The details of the variables included, as well as the outcome (life expectancy), are listed in the table below. [Smoking rates](http://apps.who.int/gho/data/node.main.1250?lang=en), [rates of insufficient physical activity](http://apps.who.int/gho/data/node.main.A893?lang=en), and [malaria](http://apps.who.int/gho/data/view.main.14111?lang=en) and [HIV](http://apps.who.int/gho/data/node.main.620?lang=en) incidence were also considered for this model, but were found to have high rates of missing data and were therefore excluded.

These analyses were performed on data extracted on the 24th July 2016.

**Description of data used for each variable**

| Variable | Year | Measure used | Gender sample | Source URL |
| -------- | ---- | ------------ | ------------- | ---------- |
| Life expectancy | 2015 | Life expectancy at birth | Both sexes | http://apps.who.int/gho/data/node.main.688?lang=en|
| Alcohol | 2010 | Per capita consumption (in litres) | All beverage types | http://apps.who.int/gho/data/node.main.A1026?lang=en|
| Overweight | 2014 | Age-standarised rate (BMI >= 25) | Both sexes | http://apps.who.int/gho/data/node.main.A897A?lang=en|
| Cholesterol | 2008 | Age-standardised rate | Both sexes | http://apps.who.int/gho/data/node.main.A884?lang=en |
| Blood sugar | 2014 | Crude rate | Both sexes | http://apps.who.int/gho/data/node.main.A869?lang=en |
| Improved water | 2015 | % using improved water sources | Total | http://apps.who.int/gho/data/node.main.167?lang=en | 
| Improved sanitation | 2015 | % using improved sanitation | Total | http://apps.who.int/gho/data/node.main.167?lang=en |
| Maternal deaths | 2015 | Maternal mortality ratio | N/A  | http://apps.who.int/gho/data/node.main.MATMORT?lang=en |
| UV radiation | 2004 | Exposure to solar UV | N/A | http://apps.who.int/gho/data/node.main.164?lang=en |
| Homicide | 2012 | Rate of homicides | N/A | http://apps.who.int/gho/data/node.main.VIOLENCEHOMICIDE?lang=en |
| Traffic deaths | 2013 | Rate of road traffic deaths | N/A | http://apps.who.int/gho/data/node.main.A997?lang=en |
| Tuberculosis | 2014 | Incidence of tuberculosis | All cases | http://apps.who.int/gho/data/view.main.57040ALL?lang=en |
| Suicide | 2012 | Age-standardised rate | Both sexes | http://apps.who.int/gho/data/node.main.MHSUICIDE?lang=en |

The variables for this analysis are extracted directly from the website below:

In [ ]:
def dataImport(dataurl):
	url = dataurl
	return pd.read_csv(url)

life = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHOSIS_000001,WHOSIS_000015&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;SEX")
alcohol = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/SA_0000001400&profile=crosstable&filter=COUNTRY:*;YEAR:2015;YEAR:2014;YEAR:2013;YEAR:2012;YEAR:2011;YEAR:2010;YEAR:2009;YEAR:2008;YEAR:2007;YEAR:2006;YEAR:2005;YEAR:2004;YEAR:2003;YEAR:2002;YEAR:2001;YEAR:2000&x-sideaxis=COUNTRY;DATASOURCE;ALCOHOLTYPE&x-topaxis=GHO;YEAR")
oweight = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_BMI_25A&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;AGEGROUP;SEX")
chol = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/CHOL_01,CHOL_02&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX")
bsugar = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/NCD_GLUC_03,NCD_GLUC_04&profile=crosstable&filter=AGEGROUP:*;COUNTRY:*;SEX:*&x-sideaxis=COUNTRY;YEAR;AGEGROUP&x-topaxis=GHO;SEX")
sanitation = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/WHS5_122,WHS5_158&profile=crosstable&filter=COUNTRY:*;RESIDENCEAREATYPE:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO;RESIDENCEAREATYPE")
maternal = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000026,MORT_MATERNALNUM&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO")
uvrad = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/UV_1&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR")
homicides = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/VIOLENCE_HOMICIDENUM,VIOLENCE_HOMICIDERATE&profile=crosstable&filter=COUNTRY:*;AGEGROUP:-;SEX:-&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR")
traffdeath = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/RS_196,RS_198&profile=crosstable&filter=COUNTRY:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR")
tb = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MDG_0000000020,TB_e_inc_num,TB_e_inc_tbhiv_100k,TB_e_inc_tbhiv_num&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY;YEAR&x-topaxis=GHO")
suicide = dataImport("http://apps.who.int/gho/athena/data/xmart.csv?target=GHO/MH_12&profile=crosstable&filter=COUNTRY:*;REGION:*&x-sideaxis=COUNTRY&x-topaxis=GHO;YEAR;SEX")

## Cleaning the data

Each of the variables have been imported in separate DataFrames that need to be cleaned. These need to be cleaned so that only the relevant years and variables are remaining. In addition, some of the DataFrames have columns that contain confidence intervals, so need to be parsed so that only the main number remains.

In [ ]:
def cleaningData(data, rowsToKeep, outcome, colsToDrop = [], varNames = [], colsToConvert = [], year = None):
    d = data.ix[rowsToKeep : ]
    if colsToDrop:
        d = d.drop(d.columns[colsToDrop], axis = 1)
    
    d.columns = varNames
    
    if (d[outcome].dtype == 'O'):
        if (d[outcome].str.contains("\[").any()):
            d[outcome] = d[outcome].apply(lambda x: x.split(' [')[0])
            d[outcome] = d[outcome].str.replace(' ', '')
    
    d[colsToConvert] = d[colsToConvert].apply(lambda x: pd.to_numeric(x, errors ='coerce'))
    
    if 'Year' in list(d.columns.values):
        d = d.loc[d['Year'] == year]
        del d['Year']
    return d

# Clean each DataFrame below. Note that 'alcohol' required 2 additional cleaning steps as it
# had a very different sturcture from the other data sources.

life = cleaningData(life, 1, 'LifeExpectancy', range(3, 8), ['Country', 'Year', 'LifeExpectancy'],
					['Year', 'LifeExpectancy'], 2015)

alcohol = cleaningData(alcohol, 1, 'Alcohol', [1] + range(3, 8) + range(9, 19), ['Country', 'Type', 'Alcohol'],
                    ['Alcohol'])
alcohol = alcohol[alcohol['Type'].str.contains("All types")]
del alcohol['Type']

oweight = cleaningData(oweight, 3, 'Overweight', range(2, 7), ['Country', 'Overweight'], 
					  ['Overweight'])
chol = cleaningData(chol, 1, 'Cholesterol', [2, 4, 5, 6, 7, 8], ['Country', 'Year', 'Cholesterol'],
                    ['Year', 'Cholesterol'], 2008)
bsugar = cleaningData(bsugar, 1, 'BloodSugar', [2, 4, 5, 6, 7], ['Country', 'Year', 'BloodSugar'],
                    ['Year', 'BloodSugar'], 2014)
sanitation = cleaningData(sanitation, 1, 'ImprovedWater', [2, 3, 5, 6], 
                           ['Country', 'Year', 'ImprovedWater', 'ImprovedSanitation'],
                           ['Year', 'ImprovedWater', 'ImprovedSanitation'], 2015)
maternal = cleaningData(maternal, 0, 'MaternalDeaths', [3], ['Country', 'Year', 'MaternalDeaths'],
                           ['Year', 'MaternalDeaths'], 2015)
uvrad = cleaningData(uvrad, 1, 'UVRadiation', [], ['Country', 'UVRadiation'], 
                      ['UVRadiation'])
homicides = cleaningData(homicides, 1, 'HomicideRate', [1], ['Country', 'HomicideRate'],
                    ['HomicideRate'])
traffdeath = cleaningData(traffdeath, 1, 'TrafficDeaths', [1], ['Country', 'TrafficDeaths'],
                    ['TrafficDeaths'])
tb = cleaningData(tb, 0, 'Tubercululosis', [2, 4, 5], ['Country', 'Year', 'Tubercululosis'],
                    ['Year', 'Tubercululosis'], 2014)
suicide = cleaningData(suicide, 2, 'Suicide', [2, 3], ['Country', 'Suicide'],
                    ['Suicide'])

## Creating the final dataset

Finally, we need to merge all of the datasets. There are also a small number of missing values. As the LASSO regression we will use below cannot run on a dataset with missing values, these need to be deleted.

In [ ]:
# Now that the data is cleaned, time to merge all of the variables into one DataFrame.
def mergeFunc(dataframe1, dataframe2):
    return pd.merge(dataframe1, dataframe2, left_on='Country', 
                    right_on='Country', how='outer')

totaldf = mergeFunc(life, alcohol)
for i in [oweight, chol, bsugar, sanitation, maternal,
         uvrad, homicides, traffdeath, tb, suicide]:
    totaldf = mergeFunc(totaldf, i)

# Delete all rows with missing values from full DataFrame
totaldf = totaldf.dropna()

## Preparing the dataset for linear LASSO regression

The data were screened to see if they met the assumptions of linear regression.
* The outcome was negatively skewed, so an inverse square root transformation was applied to normalise it. This somewhat corrected the skew, but note that it is not completely normal.
* There were a two pairs of predictors with potential multicollinearity issues (correlations of above 0.8); however, a sensitivity analysis demonstrated that having them in the final model together did not affect the coefficients:
     * ImprovedSanitation and ImprovedWater
     * ImprovedSanitation and Cholesterol
* As recommended for LASSO regression, all of the predictors were standardised prior to adding them to the model.

Finally, ridge regression was considered, but as I wanted to reduce the number of predictors I ultimately decided on LASSO regression to fit the model.

In [ ]:
# Transform the outcome (life expectancy) and make new variable ('TransformedLife')
totaldf['TransformedLife'] = np.sqrt((max(totaldf['LifeExpectancy']) + 1) - totaldf['LifeExpectancy'])

# Standardise the predictors
for i in list(totaldf.columns.values)[2:14]:
    totaldf['%s' % i] = (totaldf['%s' % i] - totaldf['%s' % i].mean()) / totaldf['%s' %i].std()

## Evaluating the correct alpha level

The method of evaluating the fit of LASSO regressions at various levels of alpha was taken from [this blog post](http://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/). The model I selected was at an alpha of 0.001, as it was the most parsimonious model with the lowest RSS. Below is the code I adapted from this blog post, which produces a table of the coefficients and RSS's at each of 10 levels of alpha.

In [ ]:
def lasso_regression(data, predictors, alpha):
    #Fit the model
    lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e5)
    lassoreg.fit(data[predictors],data['TransformedLife'])
    y_pred = lassoreg.predict(data[predictors])
    
    #Return the result in pre-defined format
    rss = sum((y_pred-data['TransformedLife'])**2)
    ret = [rss]
    ret.extend([lassoreg.intercept_])
    ret.extend(lassoreg.coef_)
    return ret

predictors = list(totaldf.columns.values)[2:14]

alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

col = ['rss','intercept'] + list(totaldf.columns.values)[2:14]
ind = ['alpha_%.2g'%alpha_lasso[i] for i in range(0,10)]
coef_matrix_lasso = pd.DataFrame(index=ind, columns=col)

for i in range(10):
    coef_matrix_lasso.iloc[i,] = lasso_regression(totaldf, predictors, alpha_lasso[i])

coef_matrix_lasso

## Producing the results of the final LASSO model

Finally, having selected a model with an alpha of 0.001, I will print out the:
* Model coefficients
* RSS
* Predicted life expectancy.

Note that the predicted life expectancy and RSS is for the **training** data, as there was not enough sample for a test set. As such, it is likely an overfit. Also, remember that as the outcome was transformed to fit the model, the predicted values need to be back-transformed to be interpretable.

In [ ]:
# Produce the model RSS, coefficients and predicted life expectancy
lassoreg = Lasso(alpha=0.001, normalize=True, max_iter=1e5)
lassoreg.fit(totaldf[list(totaldf.columns.values)[2:14]], totaldf['TransformedLife'])
rss = sum((y_pred-totaldf['TransformedLife'])**2)
ret = [rss]
ret.extend([lassoreg.intercept_])
ret.extend(lassoreg.coef_)

# Print out the coefficients of the model:
print("Model coefficients")
print({key:value for key, value in zip(list(totaldf.columns.values)[2:14], [round(elem, 2) for elem in ret])})

# RSS:
print("\nModel RSS")
print(rss)

# Predicted life expectancy on same data
print("\nPredicted values by country")
print({key:value for key, value in zip(list(totaldf['Country']),
                                 [round(elem, 1) for elem in (max(totaldf['LifeExpectancy']) + 1) - (y_pred**2)])})